In [1]:
!pip install --no-cache-dir -q transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 106.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 163.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 220.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 207.4 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/spbu-smart-assistant/support-chatbot.git

Cloning into 'support-chatbot'...
remote: Enumerating objects: 587, done.
remote: Counting objects: 100% (243/243), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 587 (delta 138), reused 229 (delta 127), pack-reused 344
Receiving objects: 100% (587/587), 403.01 MiB | 16.28 MiB/s, done.
Resolving deltas: 100% (210/210), done.
Updating files: 100% (156/156), done.


In [1]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-absum'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model.cuda() if torch.cuda.is_available() else None
model.eval();

def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0, 
    **kwargs
):
    """
    Summarize the text
    The following parameters are mutually exclusive:
    - n_words (int) is an approximate number of words to generate.
    - compression (float) is an approximate length ratio of summary and original text.
    """
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from datasets import load_dataset

In [4]:
our_dataset = load_dataset('json', data_files='support-chatbot/data/raw/summary_data.json')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-e2acaad298041a60/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
our_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summa'],
        num_rows: 16
    })
})

In [6]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summa"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
tokenized_our_dataset = our_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [8]:
tokenized_our_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summa', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 16
    })
})

In [9]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=MODEL_NAME)

In [10]:
!pip install -q rouge_score evaluate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.7 MB/s eta 0:00:00


In [11]:
import evaluate

rouge = evaluate.load("rouge")

In [12]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    print('True:', decoded_labels)
    print('prediction:', decoded_preds)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [13]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [20]:
#model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_model",
    evaluation_strategy="steps",
    eval_steps=20,
    learning_rate=2e-5,
    per_device_train_batch_size=1, # было 4
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=50,
    predict_with_generate=True,
    push_to_hub=False,
    logging_steps=20,
)

In [21]:
tokenized_our_dataset['train']

Dataset({
    features: ['text', 'summa', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 16
})

In [22]:
from datasets import Dataset
from tqdm.auto import tqdm
for i in tqdm(range(16)):
  model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

  ex_text = [tokenized_our_dataset['train'][i]['text']]
  ex_summa = [tokenized_our_dataset['train'][i]['summa']]
  ex_input_ids = [tokenized_our_dataset['train'][i]['input_ids']]
  ex_attention_mask = [tokenized_our_dataset['train'][i]['attention_mask']]
  ex_labels = [tokenized_our_dataset['train'][i]['labels']]

  test_dataset = Dataset.from_dict({'text': ex_text, 'summa': ex_summa, 
                                    'input_ids': ex_input_ids, 
                                    'attention_mask': ex_attention_mask,
                                    'labels': ex_labels})
  
  train_text = tokenized_our_dataset['train'][:i]['text'] + tokenized_our_dataset['train'][i+1:]['text']
  train_summa = tokenized_our_dataset['train'][:i]['summa'] + tokenized_our_dataset['train'][i+1:]['summa']
  train_input_ids = tokenized_our_dataset['train'][:i]['input_ids'] + tokenized_our_dataset['train'][i+1:]['input_ids']
  train_attention_mask = tokenized_our_dataset['train'][:i]['attention_mask'] + tokenized_our_dataset['train'][i+1:]['attention_mask']
  train_labels = tokenized_our_dataset['train'][:i]['labels'] + tokenized_our_dataset['train'][i+1:]['labels']

  train_dataset = Dataset.from_dict({'text': train_text, 'summa': train_summa, 
                                    'input_ids': train_input_ids, 
                                    'attention_mask': train_attention_mask,
                                    'labels': train_labels})
  trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    )
  trainer.train()
  del model, trainer, train_dataset, test_dataset
  torch.cuda.empty_cache()
  

  0%|          | 0/16 [00:00<?, ?it/s]

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
20,1.710600,0.904814,0.000000,0.000000,0.000000,0.000000,18.000000
40,1.334400,0.778441,0.000000,0.000000,0.000000,0.000000,18.000000
60,0.835400,0.680560,0.000000,0.000000,0.000000,0.000000,13.000000
80,0.651700,0.618375,0.000000,0.000000,0.000000,0.000000,13.000000
100,0.816100,0.593361,0.000000,0.000000,0.000000,0.000000,13.000000
120,0.512400,0.588665,0.000000,0.000000,0.000000,0.000000,9.000000
140,0.498100,0.567931,0.000000,0.000000,0.000000,0.000000,9.000000
160,0.279100,0.566925,0.000000,0.000000,0.000000,0.000000,9.000000
180,0.544200,0.560118,0.000000,0.000000,0.000000,0.000000,11.000000
200,0.436800,0.556757,0.000000,0.000000,0.000000,0.000000,19.000000


True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.']
prediction: ['Спасибо, техническая поддержка спбгу. Слушайте вас.']
True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.']
prediction: ['Спасибо, техническая поддержка спбгу. Слушайте вас.']
True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.']
prediction: ['Спасибо, техническая поддержка спбгу.']
True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.']
prediction: ['Спасибо, техническая поддержка спбгу.']
True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.']
prediction: ['Спасибо, техническая поддержка спбгу.']
True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.']
prediction: ['Забыл пароль от почты.']
True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.']
prediction: ['Забыл пароль от почты.']
True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.']
prediction: ['Забыл пароль от почты.']
True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.']


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
20,1.654700,0.551621,0.000000,0.000000,0.000000,0.000000,16.000000
40,1.359600,0.379083,0.000000,0.000000,0.000000,0.000000,16.000000
60,0.955500,0.415115,0.000000,0.000000,0.000000,0.000000,17.000000
80,0.653400,0.482015,0.000000,0.000000,0.000000,0.000000,17.000000
100,0.818400,0.563142,0.000000,0.000000,0.000000,0.000000,17.000000
120,0.513200,0.591784,0.000000,0.000000,0.000000,0.000000,17.000000
140,0.534500,0.588377,0.000000,0.000000,0.000000,0.000000,17.000000
160,0.299800,0.597637,0.000000,0.000000,0.000000,0.000000,17.000000
180,0.571600,0.602413,0.000000,0.000000,0.000000,0.000000,17.000000
200,0.431500,0.628482,0.000000,0.000000,0.000000,0.000000,17.000000


...

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
20,1.593400,1.609958,0.000000,0.000000,0.000000,0.000000,19.000000
40,1.451500,1.414514,0.000000,0.000000,0.000000,0.000000,19.000000
60,0.796100,1.260667,0.000000,0.000000,0.000000,0.000000,19.000000
80,0.627000,1.344335,0.000000,0.000000,0.000000,0.000000,19.000000
100,0.788800,1.341041,0.000000,0.000000,0.000000,0.000000,19.000000
120,0.441800,1.357858,0.000000,0.000000,0.000000,0.000000,19.000000
140,0.383900,1.352290,0.000000,0.000000,0.000000,0.000000,19.000000
160,0.501200,1.335147,0.000000,0.000000,0.000000,0.000000,19.000000
180,0.472100,1.321859,0.000000,0.000000,0.000000,0.000000,19.000000
200,0.371600,1.430409,0.000000,0.000000,0.000000,0.000000,19.000000


True: ['У пользователя проблемы с подключением к дистанционному экзамену в тимс. Он не начался вовремя.']
prediction: ['У меня проблемы с подключением к дистанционному экзамену в Тимс.']
True: ['У пользователя проблемы с подключением к дистанционному экзамену в тимс. Он не начался вовремя.']
prediction: ['У меня проблемы с подключением к дистанционному экзамену в Тимс.']
True: ['У пользователя проблемы с подключением к дистанционному экзамену в тимс. Он не начался вовремя.']
prediction: ['У пользователя проблемы с подключением к дистанционному экзамену в тимс.']
True: ['У пользователя проблемы с подключением к дистанционному экзамену в тимс. Он не начался вовремя.']
prediction: ['У пользователя проблемы с подключением к дистанционному экзамену в тимс.']
True: ['У пользователя проблемы с подключением к дистанционному экзамену в тимс. Он не начался вовремя.']
prediction: ['У пользователя проблемы с подключением к дистанционному экзамену в тимс.']
True: ['У пользователя проблемы с подключ

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
20,1.573800,0.918314,0.000000,0.000000,0.000000,0.000000,4.000000
40,1.363200,0.762707,0.000000,0.000000,0.000000,0.000000,18.000000
60,0.842100,0.656685,0.000000,0.000000,0.000000,0.000000,15.000000
80,0.673200,0.615900,0.000000,0.000000,0.000000,0.000000,19.000000
100,0.817500,0.604262,0.000000,0.000000,0.000000,0.000000,19.000000
120,0.480500,0.545994,0.000000,0.000000,0.000000,0.000000,19.000000
140,0.463300,0.481360,0.000000,0.000000,0.000000,0.000000,19.000000
160,0.494100,0.449613,0.000000,0.000000,0.000000,0.000000,19.000000
180,0.447500,0.407596,0.000000,0.000000,0.000000,0.000000,19.000000
200,0.421500,0.373657,0.000000,0.000000,0.000000,0.000000,19.000000


True: ['Пользователь хочет использовать матлаб в своей работе.']
prediction: ['Спасибо.']
True: ['Пользователь хочет использовать матлаб в своей работе.']
prediction: ['Для использования программного обеспечения Matlab в своей работе.']
True: ['Пользователь хочет использовать матлаб в своей работе.']
prediction: ['Программа с открытым исходным кодом Scilab.']
True: ['Пользователь хочет использовать матлаб в своей работе.']
prediction: ['В СПбГУ предложена программа с открытым исходным кодом Scilab.']
True: ['Пользователь хочет использовать матлаб в своей работе.']
prediction: ['В СПбГУ предложена программа с открытым исходным кодом Scilab.']
True: ['Пользователь хочет использовать матлаб в своей работе.']
prediction: ['В СПбГУ предложена программа с открытым исходным кодом Scilab.']
True: ['Пользователь хочет использовать матлаб в своей работе.']
prediction: ['В СПбГУ предложена программа с открытым исходным кодом Scilab.']
True: ['Пользователь хочет использовать матлаб в своей работе.